# Librerias

In [10]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold

In [4]:
# ====================================================
# I. CLEANING AND SET-UP
# ====================================================


# ------------------------------
# 0. Load data
# ------------------------------
file_path = r"..\..\r\input\penn_jae.dat"

# Detect delimiter automatically
df_raw = pd.read_csv(file_path, sep=r"\s+", engine="python")
print("✓ Data loaded successfully")
print(df_raw.head(), "\n")

✓ Data loaded successfully
    abdt  tg  inuidur1  inuidur2  female  black  hispanic  othrace  dep  q1  \
0  10824   0        18        18       0      0         0        0    2   0   
1  10635   2         7         3       0      0         0        0    0   0   
2  10551   5        18         6       1      0         0        0    0   0   
3  10824   0         1         1       0      0         0        0    0   0   
4  10747   0        27        27       0      0         0        0    0   0   

   ...  q5  q6  recall  agelt35  agegt54  durable  nondurable  lusd  husd  \
0  ...   1   0       0        0        0        0           0     0     1   
1  ...   0   0       0        1        0        0           0     1     0   
2  ...   0   0       1        0        1        0           0     0     0   
3  ...   1   0       0        0        0        0           0     1     0   
4  ...   0   0       0        0        0        0           0     1     0   

   muld  
0     0  
1     0  
2    

In [15]:
df = df_raw.loc[df_raw["tg"].isin([0, 4])].copy()
df["T4"] = (df["tg"] == 4).astype(int)

if "inuidur1" in df.columns:
    df["y"] = np.log(df["inuidur1"])
elif "inuidur2" in df.columns:
    df["y"] = np.log(df["inuidur2"])
elif "inuidur" in df.columns:
    df["y"] = np.log(df["inuidur"])
else:
    raise ValueError("No se encontró la variable de ingreso.")

df["dep_0"] = (df["dep"] == 0).astype(int)
df["dep_1"] = (df["dep"] == 1).astype(int)
df["dep_2"] = (df["dep"] == 2).astype(int)

x_vars = [
    'female', 'black', 'othrace', 'dep_1', 'dep_2',
    'q2', 'q3', 'q4', 'q5', 'q6',
    'recall', 'agelt35', 'agegt54',
    'durable', 'nondurable', 'lusd', 'husd'
]

x_vars = [v for v in x_vars if v in df.columns]

X = df[x_vars].to_numpy()
y = df["y"].to_numpy()
d = df["T4"].to_numpy()

# II. Parte 2


In [16]:
def dml(y, d, X, n_folds=5, method_y="ols", method_d="ols"):

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=123)
    thetas, ses = [], []

    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        d_train, d_test = d[train_idx], d[test_idx]

        # Modelo para E[Y|X]
        if method_y == "ols":
            mdl_y = LinearRegression().fit(X_train, y_train)
            y_hat = mdl_y.predict(X_test)

        elif method_y == "lasso":
            mdl_y = LassoCV(cv=5).fit(X_train, y_train)
            y_hat = mdl_y.predict(X_test)

        elif method_y == "rf":
            mdl_y = RandomForestRegressor().fit(X_train, y_train)
            y_hat = mdl_y.predict(X_test)

        # Modelo para E[D|X]
        if method_d == "ols":
            mdl_d = LinearRegression().fit(X_train, d_train)
            d_hat = mdl_d.predict(X_test)

        elif method_d == "lasso":
            mdl_d = LassoCV(cv=5).fit(X_train, d_train)
            d_hat = mdl_d.predict(X_test)

        elif method_d == "rf":
            mdl_d = RandomForestClassifier().fit(X_train, d_train)
            d_hat = mdl_d.predict_proba(X_test)[:, 1]

        # Residuos
        v_hat = y_test - y_hat
        u_hat = d_test - d_hat

        # Parámetro causal
        theta = np.mean(u_hat * v_hat) / np.mean(u_hat * d_test)
        thetas.append(theta)

        # Error estándar
        se = np.std(u_hat * v_hat) / (np.sqrt(len(test_idx)) * abs(np.mean(u_hat * d_test)))
        ses.append(se)

    return {
        "theta_hat": np.mean(thetas),
        "se_hat": np.mean(ses),
        "theta_folds": thetas
    }

# ---- Estimaciones DML usando varios métodos ----

results_dml = []

res_ols = dml(y, d, X, method_y="ols", method_d="ols")
results_dml.append(("OLS", res_ols["theta_hat"], res_ols["se_hat"]))

res_lasso = dml(y, d, X, method_y="lasso", method_d="lasso")
results_dml.append(("Lasso", res_lasso["theta_hat"], res_lasso["se_hat"]))

res_rf = dml(y, d, X, method_y="rf", method_d="rf")
results_dml.append(("RandomForest", res_rf["theta_hat"], res_rf["se_hat"]))

# Resultado final en DataFrame

results_dml = pd.DataFrame(results_dml, columns=["Method", "Estimate", "SE"])
results_dml 


,Method,Estimate,SE
0,OLS,-0.068252,0.078919
1,Lasso,-0.070045,0.078839
2,RandomForest,-0.091294,0.091434


In [18]:
# ====================================================
# III. DEFINE Y, D, X
# ====================================================

# Y: Outcome (definido en la limpieza como log(inuidur1))
Y = df["y"].values

# D: Treatment
D = df["T4"].values

# X: Covariates definidos en tu parte I
x_vars = [
    'female', 'black', 'othrace',
    'dep_1', 'dep_2',
    'q2', 'q3', 'q4', 'q5', 'q6',
    'recall', 'agelt35', 'agegt54',
    'durable', 'nondurable', 'lusd', 'husd'
]

# filtrar solo columnas que existen
x_vars = [col for col in x_vars if col in df.columns]

X = df[x_vars].values


In [19]:
# ====================================================
# IV. DML FUNCTION (Partially Linear Model)
# ====================================================


def dml(Y, D, X, ml_y, ml_d, K=2):

    kf = KFold(n_splits=K, shuffle=True, random_state=123)

    tilde_Y = np.zeros(len(Y))
    tilde_D = np.zeros(len(D))

    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        Y_train, Y_test = Y[train_idx], Y[test_idx]
        D_train, D_test = D[train_idx], D[test_idx]

        # Fit ML models
        ml_y.fit(X_train, Y_train)
        ml_d.fit(X_train, D_train)

        # Predict m(X) and g(X)
        m_hat = ml_y.predict(X_test)
        g_hat = ml_d.predict(X_test)

        # Residuals
        tilde_Y[test_idx] = Y_test - m_hat
        tilde_D[test_idx] = D_test - g_hat

    # Final partialling out regression
    theta = np.sum(tilde_D * tilde_Y) / np.sum(tilde_D * tilde_D)

    # Standard error
    resid = tilde_Y - theta * tilde_D
    sigma2 = np.mean(resid**2)
    se = np.sqrt(sigma2 / np.sum(tilde_D**2))

    return theta, se


In [20]:
results = []

# ---------------------------
# 1. OLS
# ---------------------------
ols_y = LinearRegression()
ols_d = LinearRegression()
theta, se = dml(Y, D, X, ols_y, ols_d)
results.append(["OLS", theta, se])


# ---------------------------
# 2. Lasso
# ---------------------------
lasso_y = LassoCV(cv=5, random_state=123)
lasso_d = LassoCV(cv=5, random_state=123)
theta, se = dml(Y, D, X, lasso_y, lasso_d)
results.append(["Lasso", theta, se])


# ---------------------------
# 3. Random Forest
# ---------------------------
rf_y = RandomForestRegressor(
    n_estimators=500,
    max_depth=None,
    random_state=123
)
rf_d = RandomForestRegressor(
    n_estimators=500,
    max_depth=None,
    random_state=123
)
theta, se = dml(Y, D, X, rf_y, rf_d)
results.append(["Random Forest", theta, se])


# ---------------------------
# 4. Neural Network (MLP)
# ---------------------------
nn_y = MLPRegressor(
    hidden_layer_sizes=(64, 64),
    activation="relu",
    max_iter=2000,
    random_state=123
)
nn_d = MLPRegressor(
    hidden_layer_sizes=(64, 64),
    activation="relu",
    max_iter=2000,
    random_state=123
)
theta, se = dml(Y, D, X, nn_y, nn_d)
results.append(["Neural Net", theta, se])


# ---------------------------
# TABLE
# ---------------------------
table_results = pd.DataFrame(results, columns=["Model", "Theta", "Std.Error"])
table_results

,Model,Theta,Std.Error
0,OLS,-0.079899,0.035138
1,Lasso,-0.076806,0.035278
2,Random Forest,-0.097995,0.035501
3,Neural Net,-0.065052,0.034899


La estimación del efecto causal del tratamiento T4 sobre la duración de desempleo (medida como log(inuidur1)) se realizó utilizando el método Debiased Machine Learning (DML) con cross-fitting. Se aplicaron cuatro modelos diferentes para aproximar las funciones condicionales necesarias: OLS, Lasso, Random Forest y una Red Neuronal (MLP).

Los resultados muestran que:

En todos los modelos, el efecto estimado (theta) es negativo, lo cual sugiere que pertenecer al grupo de tratamiento reduce la duración del desempleo en promedio.

Las estimaciones varían entre –0.098 y –0.065, dependiendo del modelo utilizado.

El error estándar es relativamente estable y cercano a 0.035 en todos los casos, indicando una incertidumbre similar entre modelos.

El modelo que predice un mayor impacto negativo es Random Forest (θ ≈ –0.098), mientras que el impacto menor proviene de la Red Neuronal (θ ≈ –0.065).

OLS y Lasso producen estimaciones muy similares entre sí, alrededor de –0.078, lo que refleja estabilidad entre métodos lineales.

En conjunto, los resultados sugieren que el tratamiento T4 está asociado con una disminución moderada en el logaritmo de la duración del desempleo. Dado que los modelos no-lineales (Random Forest y NN) ofrecen resultados más dispersos, los modelos lineales (OLS y Lasso) parecen proporcionar estimaciones más estables bajo el enfoque DML.

# III. Parte 2 Comparacion

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:

# =======================================================
#   DML SIN CROSS-FITTING
# =======================================================

def dml_no_cf(Y, D, X, ml_y, ml_d, train_fraction=0.7):

    n = len(Y)
    idx = np.arange(n)

    # Train/Test split fijo (sin cross-fitting)
    train_size = int(train_fraction * n)
    np.random.seed(123)
    train_idx = np.random.choice(idx, size=train_size, replace=False)
    test_idx = np.setdiff1d(idx, train_idx)

    # Entrenar modelos con el split único
    ml_y.fit(X[train_idx], Y[train_idx])
    ml_d.fit(X[train_idx], D[train_idx])

    # Predicciones
    y_hat_train = ml_y.predict(X[train_idx])
    y_hat_test  = ml_y.predict(X[test_idx])

    d_hat_train = ml_d.predict(X[train_idx])
    d_hat_test  = ml_d.predict(X[test_idx])

    # RMSE
    rmse_y_train = np.sqrt(mean_squared_error(Y[train_idx], y_hat_train))
    rmse_y_test  = np.sqrt(mean_squared_error(Y[test_idx],  y_hat_test))

    rmse_d_train = np.sqrt(mean_squared_error(D[train_idx], d_hat_train))
    rmse_d_test  = np.sqrt(mean_squared_error(D[test_idx],  d_hat_test))

    # Residuos en test
    v_hat = Y[test_idx] - y_hat_test
    u_hat = D[test_idx] - d_hat_test

    theta_hat = np.mean(u_hat * v_hat) / np.mean(u_hat * D[test_idx])
    se_hat = np.std(u_hat * v_hat) / (np.sqrt(len(test_idx)) * abs(np.mean(u_hat * D[test_idx])))

    return {
        "theta": theta_hat,
        "se": se_hat,
        "rmse_y_test": rmse_y_test,
        "rmse_d_test": rmse_d_test,
        "rmse_y_train": rmse_y_train,
        "rmse_d_train": rmse_d_train
    }

In [ ]:
results_no_cf = []

# -------------------------
# OLS
# -------------------------
ols_y = LinearRegression()
ols_d = LinearRegression()
res = dml_no_cf(Y, D, X, ols_y, ols_d)
results_no_cf.append(["OLS", res["theta"], res["se"], res["rmse_y_test"], res["rmse_d_test"]])

# -------------------------
# Lasso
# -------------------------
lasso_y = LassoCV(cv=5, random_state=123)
lasso_d = LassoCV(cv=5, random_state=123)
res = dml_no_cf(Y, D, X, lasso_y, lasso_d)
results_no_cf.append(["Lasso", res["theta"], res["se"], res["rmse_y_test"], res["rmse_d_test"]])

# -------------------------
# Random Forest
# -------------------------
rf_y = RandomForestRegressor(n_estimators=500, random_state=123)
rf_d = RandomForestRegressor(n_estimators=500, random_state=123)
res = dml_no_cf(Y, D, X, rf_y, rf_d)
results_no_cf.append(["Random Forest", res["theta"], res["se"], res["rmse_y_test"], res["rmse_d_test"]])

# -------------------------
# Neural Network
# -------------------------
nn_y = MLPRegressor(hidden_layer_sizes=(64, 64), max_iter=2000, random_state=123)
nn_d = MLPRegressor(hidden_layer_sizes=(64, 64), max_iter=2000, random_state=123)
res = dml_no_cf(Y, D, X, nn_y, nn_d)
results_no_cf.append(["Neural Net", res["theta"], res["se"], res["rmse_y_test"], res["rmse_d_test"]])

# Tabla final sin CF
table_no_cf = pd.DataFrame(results_no_cf, columns=["Model", "Theta_NoCF", "SE_NoCF", "RMSE_Y_Test", "RMSE_D_Test"])
table_no_cf

In [ ]:
table_no_cf2 = table_no_cf[["Model", "Theta_NoCF", "SE_NoCF"]]

In [ ]:
comparacion= pd.merge(table_results, table_no_cf2, on="Model", how="left")
comparacion

### **Análisis Parte III – Comparación entre Cross-Fitting y No Cross-Fitting**

### 1. Sobre el RMSE al predecir *y* y *d*
Al estimar sin cross-fitting, los RMSE de $\hat{y}$ y $\hat{d}$ suelen ser **menores artificialmente**, especialmente en la muestra de entrenamiento, porque los modelos se ajustan sobre los mismos datos donde serán evaluados. Con cross-fitting, las predicciones fuera de muestra reflejan el verdadero desempeño predictivo, por lo que los RMSE son más realistas y tienden a ser un poco mayores.

### 2. Razón por la cual una función produce un RMSE más bajo que la otra
La función sin cross-fitting produce un RMSE más bajo debido al **overfitting**: el modelo memoriza patrones específicos del conjunto de entrenamiento, por lo que predice muy bien en los mismos datos usados para ajustar sus parámetros. En contraste, el cross-fitting evalúa siempre sobre datos no vistos, evitando ese sesgo optimista y entregando un RMSE más honesto sobre la capacidad de generalización.

### 3. Problema de estimar sin cross-fitting
Si usamos la estimación sin cross-fitting, el overfitting se transfiere directamente al cálculo de los residuales $\tilde{Y}$ y $\tilde{D}$, lo que sesga el estimador de $\theta$. Esto rompe la propiedad clave del DML: la ortogonalidad entre los errores de predicción y la variable de tratamiento. Como consecuencia, el estimador resulta **sesgado e inconsistente**, pudiendo llevar a conclusiones